# Practica para Sistemas de Bases de Datos (Feb 21)

## Autor: Martín Romera Sobrado
## Contacto: mromera95@alumno.uned.es

### Inicialización de la base de datos

Creamos e iniciamos la base de datos si todavía no lo está.

In [1]:
! hadoop fs -mkdir librosdb

In [2]:
! hadoop fs -mkdir librosdb/authors/ && hadoop fs -mkdir librosdb/dataset

In [3]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f db/db.hql

scan complete in 18ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
que no este creada.alhost:10000/default> -- Crea la base de datos en el caso de  
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> create database if not exists librosdb
de libros'. . . . . . . . . . . . . . .> Comment 'Base de datos con información  
. . . . . . . . . . . . . . . . . . . .> Location '/home/cloudera/librosdb'
n Romera Sobrado', 'Creada. . . . . . .> With dbproperties('Creada por' = 'Martí  el' = '9-Nov-2020');
INFO  : Compiling command(queryId=hive_20201229052727_e9c15f94-d17c-4a66-96c1-69b673397e7c): create database if not exists librosdb
Comment 'Base de datos con información de libros'
Location '/home/cloudera/librosdb'
With dbproperties('Creada por' = 'Martín Romera Sobrado', 'Creada el' = '9-Nov-2020')
INFO  : Sema

Los contenidos de *db/db.hql* son los siguientes:

```HiveQL
create database if not exists librosdb
Comment 'Base de datos con información de libros'
Location '/home/cloudera/librosdb'
With dbproperties('Creada por' = 'Martín Romera Sobrado', 'Creada el' = '9-Nov-2020');
```

### Ejercicios

Ej 1. *Crea las tablas necesarias para almacenar los datos. Pueden ser internas o externas en función de los datos que se desee. La decisión de interna o externa debe estar razonada.*

Las tablas serán externas, ya que en el ejercicio 5 tendremos un programa *MapReduce* en python que acceda a los datos.

In [4]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej1.hql

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/> -- Creamos las dos tablas: authors y datasets
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052727_cad66978-5868-4a8c-8c89-ad6118c305df): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052727_cad66978-5868-4a8c-8c89-ad6118c305df); Time taken: 0.074 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052727_cad66978-5868-4a8c-8c89-ad6118c305df): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_20201229

Los contenidos de ej1.hql son los siguientes:
```HiveQL
-- Creamos las dos tablas: authors y datasets

USE librosdb;

-- Tabla authors

CREATE EXTERNAL TABLE IF NOT EXISTS authors
(
    author_id INT,
    author_name STRING
)
COMMENT 'Tabla de autores'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\;'
tblproperties("skip.header.line.count"="1"); 

-- Tabla dataset

CREATE EXTERNAL TABLE IF NOT EXISTS dataset
(
    title STRING,
    author_id INT,
    bestsellers_rank INT,
    imprint BINARY,
    publication_date STRING,
    rating_avg DOUBLE,
    rating_count INT
)
COMMENT 'Tabla de libros'
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\;'
tblproperties("skip.header.line.count"="1"); 
```

Ej 2. *Importa los datos en las tablas creadas*

In [5]:
! hadoop fs -put db/csv/authors.csv librosdb/authors

In [6]:
! hadoop fs -put db/csv/dataset.csv librosdb/dataset

In [7]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej2.hql

scan complete in 4ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
tosjdbc:hive2://localhost:10000/> -- Importamos datos de los csv a la base de da 
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052727_0344d4d5-8f27-4b26-adeb-a62c4fdd75f6): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052727_0344d4d5-8f27-4b26-adeb-a62c4fdd75f6); Time taken: 0.087 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052727_0344d4d5-8f27-4b26-adeb-a62c4fdd75f6): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_202012

Los contenidos de el archivo *db/ej2.hql*, son los siguientes:

```HiveQL
-- Importamos datos de los csv a la base de datos

USE librosdb;

LOAD DATA INPATH '/user/cloudera/librosdb/authors/authors.csv' INTO TABLE authors;

LOAD DATA INPATH '/user/cloudera/librosdb/dataset/dataset.csv' INTO TABLE dataset;
```

Los dos ficheros csv, son los que proporcionan para la práctica, sin se alterados en ningún momento.

Ej 3. *Crea una vista sobre las tablas creadas. Esta vista tendrá para cada título, el nombre del autor, fecha de publicación, y valoración media.*

In [8]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej3.hql

scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
los siguientes atributos::10000/> -- Crea una vista para las tablas creadas con  
ión y Valoración mediaost:10000/> -- Titulo, Nombre del autor, Fecha de publicac 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052727_ef8b2bef-4ec1-47d7-8e90-1c422b783f22): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052727_ef8b2bef-4ec1-47d7-8e90-1c422b783f22); Time taken: 0.073 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052727_ef8b2bef-4ec1-47d7-8e90-1c422b783f22): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : 

Los contenidos de ej3.hql son:
```HiveQL
-- Crea una vista para las tablas creadas con los siguientes atributos:
-- Titulo, Nombre del autor, Fecha de publicación y Valoración media
USE librosdb;

CREATE VIEW IF NOT EXISTS vista_libros 
(
    title COMMENT 'Titulo del libro',
    author_name COMMENT 'Autor del libro',
    publication_date COMMENT 'Fecha de publicación',
    rating_avg COMMENT 'Valoración media del libro'
)
AS SELECT title, author_name, publication_date, rating_avg
FROM authors, dataset
WHERE authors.author_id = dataset.author_id;
```

Ej 4. *Crea las consultas de hive para responder las siguientes cuestiones:*

- *¿Cuál es el título del libro que ocupa el primer lugar en el ranking de bestsellers?*

In [9]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej4_a.hql

scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
de bestsellers//localhost:10000/> -- Selecciona el libro más alto en el ranking  
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052828_8d38ecea-e7c0-4668-8222-3e34d7c41bd5): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052828_8d38ecea-e7c0-4668-8222-3e34d7c41bd5); Time taken: 0.052 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052828_8d38ecea-e7c0-4668-8222-3e34d7c41bd5): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_202012

Los contenidos de ej4_a.hql son:
```HiveQL
-- Selecciona el libro más alto en el ranking de bestsellers

USE librosdb;

SELECT title
FROM dataset LEFT SEMI JOIN (
    SELECT min(bestsellers_rank) AS bestseller
    FROM dataset
) AS a 
ON bestsellers_rank = a.bestseller
LIMIT 1;
```

- *¿Cuál es el título del libro con mejor valoración media de la autora Ana Maria Spagna?*

In [10]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej4_b.hql

scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
 Ana Maria Spagnaocalhost:10000/> -- Selecciona el libro con mejor valoración de 
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052828_b291abd8-6071-4e60-81d5-e1e58d2c063e): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052828_b291abd8-6071-4e60-81d5-e1e58d2c063e); Time taken: 0.051 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052828_b291abd8-6071-4e60-81d5-e1e58d2c063e): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_202012

Los contenidos de ej4_b.hql son:
```HiveQL
-- Selecciona el libro con mejor valoración de Ana Maria Spagna

USE librosdb;

SELECT title
FROM(
    SELECT title, rating_avg
    FROM dataset LEFT SEMI JOIN (
        SELECT max(rating_avg) AS max_rating
        FROM authors, dataset
        WHERE authors.author_id = dataset.author_id AND authors.author_name = "Ana Maria Spagna"
    ) AS b
    ON rating_avg = b.max_rating
) AS a
LIMIT 1;

```

- *¿Cuáles son los cinco autores que han escrito más libros?*

In [11]:
! beeline -u "jdbc:hive2://localhost:10000/" -f db/ej4_c.hql

scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 1.1.0-cdh5.12.0)
Driver: Hive JDBC (version 1.1.0-cdh5.12.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/> -- Selecciona los 5 autores con más libros
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> USE librosdb;
INFO  : Compiling command(queryId=hive_20201229052929_b1a15240-65e2-416c-9f30-2ab3c70ef099): USE librosdb
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20201229052929_b1a15240-65e2-416c-9f30-2ab3c70ef099); Time taken: 0.058 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=hive_20201229052929_b1a15240-65e2-416c-9f30-2ab3c70ef099): USE librosdb
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=hive_20201229052

+-----------------------+--+
|      author_name      |
+-----------------------+--+
| AA Publishing         |
| Distinctive Journals  |
| Gregory a Boyd J D    |
| Molly Elodie Rose     |
| Rand McNally          |
+-----------------------+--+
5 rows selected (76.573 seconds)
0: jdbc:hive2://localhost:10000/> 
0: jdbc:hive2://localhost:10000/> 
Closing: 0: jdbc:hive2://localhost:10000/


Los contenidos de ej4_c.hql son los siguientes:
```HiveQL
-- Selecciona los 5 autores con más libros

USE librosdb;

SELECT DISTINCT author_name
FROM authors, dataset LEFT SEMI JOIN (
    SELECT author_id, count(title) AS num_libros
    FROM dataset
    GROUP BY author_id
    ORDER BY num_libros DESC
    LIMIT 5
) AS a
ON dataset.author_id = a.author_id
WHERE authors.author_id = dataset.author_id;

```


Ej. 5. *Desarrolla el código MapReduce en Python que implemente la oprimera consulta del apartado anterior.*

La función map definida en *mapper.py*:
```python
#!/usr/bin/python
#
# Autor: Martin Romera Sobrado
# Contacto: mromera95@alumno.uned.es
# 
# Sintaxis: Python 2
#
# Funcion map para obtener el bestseller entre los libros

import sys
import csv

first_line = True
title = 0
rank = 2
bestrank = -1
bestranktitle = ""

data = sys.stdin.readlines()

for line in csv.reader(data, delimiter=';'):
    if line[rank] != "" and line[title] != "title" and line[rank] != "bestsellers-rank":
        # El libro tiene ranking
        thisrank = int(line[rank].replace('.','').replace(',',''))
        if thisrank < bestrank or bestrank == -1 :
            # Ha encontrado un libro con mejor ranking, o todavia no tenia un libro con mejor ranking
            bestrank = thisrank
            bestranktitle = line[title]

print("%s\t%s" % (str(bestrank), bestranktitle))
```

Y la función reduce definida en *reducer.py*:
```python
#!/usr/bin/python
#
# Autor: Martin Romera Sobrado
# Contacto: mromera95@alumno.uned.es
# 
# Sintaxis: Python 2
#
# Funcion reduce para obtener el bestseller entre los libros

import sys
from operator import itemgetter

bestrank = -1
besttitle = ""

for line in sys.stdin:
    line.strip()
    rank, title = line.split('\t',1)

    if bestrank == -1 or rank < bestrank:
        # Encuentra un libro con mejor ranking, o no tenia todavia un libro con mejor ranking
        bestrank = rank
        besttitle = title

    print("%s" % (besttitle))
```

In [12]:
! chmod +x mapreduce/bestseller/mapper.py mapreduce/bestseller/reducer.py

In [13]:
! hadoop fs -put db/csv/authors.csv librosdb/authors # Al llegar a este paso los archivos desaparecen del HDFS por algun motivo

In [14]:
! hadoop fs -put db/csv/dataset.csv librosdb/dataset

In [15]:
! hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \\
-file mapreduce/bestseller/mapper.py \\
-mapper ./mapreduce/bestseller/mapper.py \\
-file mapreduce/bestseller/reducer.py \\
-reducer ./mapreduce/bestseller/reducer.py \\
-input /user/cloudera/librosdb/dataset/* \\
-output /user/cloudera/librosdb/ej5-mapreduce-output 

20/12/29 05:31:03 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapreduce/bestseller/mapper.py, mapreduce/bestseller/reducer.py] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.12.0.jar] /tmp/streamjob4329337999696246688.jar tmpDir=null
20/12/29 05:31:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/12/29 05:31:05 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
20/12/29 05:31:05 INFO mapred.FileInputFormat: Total input paths to process : 1
20/12/29 05:31:05 INFO mapreduce.JobSubmitter: number of splits:2
20/12/29 05:31:05 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1604927967949_0009
20/12/29 05:31:06 INFO impl.YarnClientImpl: Submitted application application_1604927967949_0009
20/12/29 05:31:06 INFO mapreduce.Job: The url to track the job: http://quickstart.cloudera:8088/proxy/application_1604927967949_0009/
20/12/29 05:31:06 INFO mapreduce.Job: Runnin

In [17]:
!hadoop fs -text /user/cloudera/librosdb/ej5-mapreduce-output/*


Atlas Obscura, 2nd Edition : An Explorer's Guide to the World's Hidden Wonders	
	
Atlas Obscura, 2nd Edition : An Explorer's Guide to the World's Hidden Wonders	
	
